In [25]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

BASE_DIR = '../'


In [26]:
import os
import fiona
BASE_DIR = os.path.dirname(os.getcwd())
os.chdir(BASE_DIR + '/src/data/')
import util
import analysis_util
os.chdir(BASE_DIR + '/notebooks/')

In [55]:
def print_summary(total_crashes, crashes, counts, inter_count, non_inter_count):
    """                                                                                                                                                                 
    Info about intersections vs non intersections and their crash rate                                                                                                  
    """
    
    print "Total number of crashes:\t\t\t\t" + str(total_crashes)
    print "Total number of crash locations:\t\t\t\t" + str(len(crashes))
    print "Number of intersections:\t\t\t\t" + str(inter_count)
    print "Number of non-intersections:\t\t\t\t" + str(non_inter_count)
    print "Percentage of segments that are intersections:\t\t" \
        + str(round(100 * float(inter_count)/float(inter_count + non_inter_count)))
    print ""

    print "Number of intersections with at least 1 crash:\t\t" + str(counts['inter']) 
    print 'Number of intersections with more than 1 crash:\t\t' + str(counts['inter_plus'])
    print "Number of non-intersections with at least 1 crash:\t" + str(counts['non_inter'])
    print "Number of non-intersections with more than 1 crash:\t" + str(counts['non_inter_plus'])
    print ""

    total_percent = round(float(100 * (counts['inter'] + counts['non_inter']) / float(inter_count + non_inter_count)))
    print "percent of all segments with crash:\t\t\t" + str(total_percent)

    # Percentage of intersections/non-intersections                                                                                                                     
    # that have at least one crash
    inter_percent = round(float(100 * counts['inter'])/float(inter_count))
    print "percent of intersections with crash:\t\t\t" + str(inter_percent)

    non_inter_percent = round(100 * float(counts['non_inter'])/float(non_inter_count))
    print "percentage of non-intersections with crash:\t\t" + str(non_inter_percent)

    print "percentage of intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['inter_plus'])/float(inter_count)))

    print "percentage of non-intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['non_inter_plus'])/float(non_inter_count)))


In [29]:
def print_concern_summary(results):
    """                                                                                                                                                                 
    What percentage of intersections with concerns had crashes                                                                                                          
    at varying counts of concerns?                                                                                                                                      
    """

    results = results[0:20]

    display(pd.DataFrame(results, columns=[
        '# of concerns at this segment',
        '% of segments w/ v0 complaint w/ crash',
        'total # of segments with this many or more concerns',
        '% of intersections with this many or more concerns with a crash',
        '# of intersections with this many concerns',
        '% of non-intersections with this many concerns with a crash',
        '# of non-intersections with this many concerns',
        '% of these locations that were at an intersection'
    ]))

In [30]:
def print_concerns_by_type(results):
    df = pd.DataFrame(results, columns=[
        'Request type',
        '% of segments with this type of request with a crash',
        '# of segments with this type of request with a crash',
        '# of segments with this type of a request total',
        '% of intersections with this type of request with a crash',
        '# of intersections with this type of request with a crash',
        '# of intersections with this type of request total',
        '% of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request total',
    ])
    df.sort_values(by="# of segments with this type of a request total", ascending=0)
    display(df)


In [58]:
def print_analysis_for_city(dir, crash_file, concern_file, category_field='REQUESTTYPE', years=None, yearfield='CALENDAR_DATE', cutoff=100):
    inters = util.read_segments(os.path.join(dir, 'processed/maps'), get_non_inter=False)
    non_inters = util.read_segments(os.path.join(dir, 'processed/maps'), get_inter=False)
    inter_count = len(inters[0])
    non_inter_count = len(non_inters[0])

    crash_file = os.path.join(dir, 'processed', crash_file)
    concern_file = os.path.join(dir, 'processed', concern_file)
    
    crash_data, crashes = util.group_json_by_location(
        crash_file, yearfield=yearfield, years=years)
    concern_data, concerns = util.group_json_by_location(
        concern_file,
        otherfields=[category_field])

    total_crashes, crash_locations, crash_results = analysis_util.summary_crash_rate(crashes)
   
    print_summary(total_crashes, crashes, crash_results, inter_count, non_inter_count)
    total_count, total_loc, inter_total, inter_loc, non_inter_total, non_inter_loc, concern_summary = analysis_util.summary_concern_counts(
        crashes, concerns)
    print "total concerns:" + str(len(concern_data))
    print "concern locations:" + str(len(concerns))
    results = analysis_util.concern_percentages(concern_summary)
    print_concern_summary(results)
 
    concerns_by_type = analysis_util.concern_counts_by_type(
        concern_data, crashes, category_field)
    concern_percent_by_type = analysis_util.concern_percentages_by_type(
        concerns_by_type, cutoff)
    print_concerns_by_type(concern_percent_by_type)


In [59]:
#print_analysis_for_city('../osm-data', 'crash_joined.json', 'concern_joined.json', years=[2016])
#print "-------------------------------"
print_analysis_for_city('../osm-data', 
                        'crash_joined.json', 'see_click_fix_joined.json', category_field='type', years=[2016], cutoff=20)

Read in 7845 intersection, 0 non-intersection segments
Read in 0 intersection, 15434 non-intersection segments
Total number of crashes:				3202
Total number of crash locations:				1958
Number of intersections:				7845
Number of non-intersections:				15434
Percentage of segments that are intersections:		34.0

Number of intersections with at least 1 crash:		1110
Number of intersections with more than 1 crash:		397
Number of non-intersections with at least 1 crash:	847
Number of non-intersections with more than 1 crash:	152

percent of all segments with crash:			8.0
percent of intersections with crash:			14.0
percentage of non-intersections with crash:		5.0
percentage of intersections with more than 1 crash:	5.0
percentage of non-intersections with more than 1 crash:	1.0
total concerns:4866
concern locations:2733


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,17.0,2733,30.0,888,11.0,1845,32.0
1,2,21.0,836,34.0,263,15.0,573,31.0
2,3,26.0,350,40.0,112,20.0,238,32.0
3,4,29.0,190,44.0,55,24.0,135,29.0
4,5,29.0,117,39.0,33,25.0,84,28.0
5,6,26.0,82,36.0,22,22.0,60,27.0
6,7,32.0,57,39.0,18,28.0,39,32.0
7,8,37.0,41,45.0,11,33.0,30,27.0
8,9,35.0,34,38.0,8,35.0,26,24.0
9,10,43.0,21,40.0,5,44.0,16,24.0


,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,,19.0,202,1054,33.0,109,329,13.0,93,725
1,Abandoned Vehicle,19.0,5,27,17.0,1,6,19.0,4,21
2,Shovel Request,10.0,6,61,13.0,2,16,9.0,4,45
3,Other,20.0,157,792,33.0,86,263,13.0,71,529
4,Traffic Signal,42.0,52,124,62.0,47,76,10.0,5,48
5,Damaged Sign,22.0,17,77,30.0,12,40,14.0,5,37
6,Housing Available,9.0,26,281,15.0,7,48,8.0,19,233
7,Illegal Parking,18.0,30,171,28.0,15,54,13.0,15,117
8,Tree Pruning,14.0,3,22,38.0,3,8,0.0,0,14
9,Overflowing Trash Can,30.0,6,20,30.0,3,10,30.0,3,10


In [60]:
print_analysis_for_city('../osm-cambridge', 'crash_joined.json', 'concern_joined.json', category_field='issue_type', years=[2015], yearfield='Date Time')
print_analysis_for_city('../osm-cambridge', 'crash_joined.json', 'concern_joined.json', category_field='issue_type', years=[2016], yearfield='Date Time')



Read in 1287 intersection, 0 non-intersection segments
Read in 0 intersection, 2456 non-intersection segments
Total number of crashes:				1650
Total number of crash locations:				577
Number of intersections:				1287
Number of non-intersections:				2456
Percentage of segments that are intersections:		34.0

Number of intersections with at least 1 crash:		171
Number of intersections with more than 1 crash:		41
Number of non-intersections with at least 1 crash:	405
Number of non-intersections with more than 1 crash:	128

percent of all segments with crash:			15.0
percent of intersections with crash:			13.0
percentage of non-intersections with crash:		16.0
percentage of intersections with more than 1 crash:	3.0
percentage of non-intersections with more than 1 crash:	5.0
total concerns:24596
concern locations:3156


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,17.0,3156,14.0,1157,19.0,1999,37.0
1,2,19.0,2710,15.0,989,21.0,1721,36.0
2,3,20.0,2323,16.0,835,23.0,1488,36.0
3,4,22.0,1969,17.0,694,25.0,1275,35.0
4,5,23.0,1664,17.0,564,26.0,1100,34.0
5,6,24.0,1414,17.0,462,27.0,952,33.0
6,7,25.0,1205,17.0,378,29.0,827,31.0
7,8,27.0,1014,19.0,307,30.0,707,30.0
8,9,28.0,859,19.0,244,31.0,615,28.0
9,10,29.0,720,20.0,198,33.0,522,28.0


,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,Missed Trash Pickup,26.0,113,440,20.0,19,93,27.0,94,347
1,Sidewalk Defect,23.0,222,959,17.0,52,300,26.0,170,659
2,Traffic Sign Complaint,22.0,134,611,18.0,51,288,26.0,83,323
3,Other,23.0,282,1226,15.0,68,447,27.0,214,779
4,Streetlight Defect,26.0,88,339,18.0,21,118,30.0,67,221
5,Missed Compost Pickup (Green Bin),16.0,23,141,9.0,3,32,18.0,20,109
6,Tree Pruning Request,25.0,136,537,17.0,24,144,28.0,112,393
7,DPW Bike Tagged (Internal),27.0,45,165,20.0,12,60,31.0,33,105
8,Abandoned Bicycle,28.0,68,240,20.0,16,82,33.0,52,158
9,Bike Lane Obstruction,32.0,72,228,29.0,25,86,33.0,47,142


Read in 1287 intersection, 0 non-intersection segments
Read in 0 intersection, 2456 non-intersection segments
Total number of crashes:				1588
Total number of crash locations:				542
Number of intersections:				1287
Number of non-intersections:				2456
Percentage of segments that are intersections:		34.0

Number of intersections with at least 1 crash:		145
Number of intersections with more than 1 crash:		38
Number of non-intersections with at least 1 crash:	396
Number of non-intersections with more than 1 crash:	126

percent of all segments with crash:			14.0
percent of intersections with crash:			11.0
percentage of non-intersections with crash:		16.0
percentage of intersections with more than 1 crash:	3.0
percentage of non-intersections with more than 1 crash:	5.0
total concerns:24596
concern locations:3156


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,16.0,3156,12.0,1157,19.0,1999,37.0
1,2,18.0,2710,13.0,989,21.0,1721,36.0
2,3,19.0,2323,13.0,835,23.0,1488,36.0
3,4,20.0,1969,14.0,694,24.0,1275,35.0
4,5,22.0,1664,15.0,564,26.0,1100,34.0
5,6,23.0,1414,14.0,462,28.0,952,33.0
6,7,24.0,1205,14.0,378,29.0,827,31.0
7,8,25.0,1014,15.0,307,30.0,707,30.0
8,9,27.0,859,14.0,244,32.0,615,28.0
9,10,29.0,720,16.0,198,34.0,522,28.0


,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,Missed Trash Pickup,24.0,107,440,14.0,13,93,27.0,94,347
1,Sidewalk Defect,24.0,229,959,17.0,50,300,27.0,179,659
2,Traffic Sign Complaint,20.0,122,611,14.0,40,288,25.0,82,323
3,Other,21.0,259,1226,11.0,51,447,27.0,208,779
4,Streetlight Defect,21.0,71,339,12.0,14,118,26.0,57,221
5,Missed Compost Pickup (Green Bin),22.0,31,141,13.0,4,32,25.0,27,109
6,Tree Pruning Request,22.0,117,537,17.0,24,144,24.0,93,393
7,DPW Bike Tagged (Internal),31.0,51,165,18.0,11,60,38.0,40,105
8,Abandoned Bicycle,26.0,63,240,20.0,16,82,30.0,47,158
9,Bike Lane Obstruction,31.0,70,228,16.0,14,86,39.0,56,142


In [61]:
print_analysis_for_city('../osm-dc', 'crash_joined.json', 'concern_joined.json', years=[2015])
print_analysis_for_city('../osm-dc', 'crash_joined.json', 'concern_joined.json', years=[2016])


Read in 7480 intersection, 0 non-intersection segments
Read in 0 intersection, 14901 non-intersection segments
Total number of crashes:				19694
Total number of crash locations:				7150
Number of intersections:				7480
Number of non-intersections:				14901
Percentage of segments that are intersections:		33.0

Number of intersections with at least 1 crash:		2121
Number of intersections with more than 1 crash:		1005
Number of non-intersections with at least 1 crash:	5028
Number of non-intersections with more than 1 crash:	2602

percent of all segments with crash:			32.0
percent of intersections with crash:			28.0
percentage of non-intersections with crash:		34.0
percentage of intersections with more than 1 crash:	13.0
percentage of non-intersections with more than 1 crash:	17.0
total concerns:5357
concern locations:3342


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,53.0,3342,50.0,1652,56.0,1690,49.0
1,2,65.0,1023,63.0,531,67.0,492,52.0
2,3,73.0,422,73.0,245,74.0,177,58.0
3,4,81.0,206,78.0,129,87.0,77,63.0
4,5,83.0,115,76.0,74,98.0,41,64.0
5,6,85.0,67,78.0,46,100.0,21,69.0
6,7,86.0,43,80.0,30,100.0,13,70.0
7,8,87.0,30,81.0,21,100.0,9,70.0
8,9,87.0,23,81.0,16,100.0,7,70.0
9,10,87.0,15,80.0,10,100.0,5,67.0


,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,Speeding,52.0,375,724,48.0,107,223,53.0,268,501
1,Other Driving Issue,64.0,348,547,62.0,163,263,65.0,185,284
2,Jaywalking,78.0,121,156,80.0,71,89,75.0,50,67
3,Other Walking Issue,54.0,175,323,52.0,82,158,56.0,93,165
4,Cyclist behavior,60.0,67,111,55.0,33,60,67.0,34,51
5,Double parking,71.0,142,199,62.0,16,26,73.0,126,173
6,Accessibility Issue,53.0,78,146,55.0,35,64,52.0,43,82
7,Failure to stop for pedestrians,53.0,322,612,51.0,234,455,56.0,88,157
8,Red light running,68.0,201,294,71.0,154,216,60.0,47,78
9,Blocking the crosswalk,73.0,88,121,81.0,68,84,54.0,20,37


Read in 7480 intersection, 0 non-intersection segments
Read in 0 intersection, 14901 non-intersection segments
Total number of crashes:				28801
Total number of crash locations:				8797
Number of intersections:				7480
Number of non-intersections:				14901
Percentage of segments that are intersections:		33.0

Number of intersections with at least 1 crash:		3074
Number of intersections with more than 1 crash:		1893
Number of non-intersections with at least 1 crash:	5722
Number of non-intersections with more than 1 crash:	3231

percent of all segments with crash:			39.0
percent of intersections with crash:			41.0
percentage of non-intersections with crash:		38.0
percentage of intersections with more than 1 crash:	25.0
percentage of non-intersections with more than 1 crash:	22.0
total concerns:5357
concern locations:3342


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,66.0,3342,68.0,1652,63.0,1690,49.0
1,2,78.0,1023,82.0,531,72.0,492,52.0
2,3,83.0,422,87.0,245,77.0,177,58.0
3,4,91.0,206,89.0,129,94.0,77,63.0
4,5,95.0,115,93.0,74,98.0,41,64.0
5,6,94.0,67,91.0,46,100.0,21,69.0
6,7,95.0,43,93.0,30,100.0,13,70.0
7,8,97.0,30,95.0,21,100.0,9,70.0
8,9,96.0,23,94.0,16,100.0,7,70.0
9,10,100.0,15,100.0,10,100.0,5,67.0


,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,Speeding,63.0,458,724,66.0,147,223,62.0,311,501
1,Other Driving Issue,76.0,415,547,78.0,205,263,74.0,210,284
2,Jaywalking,85.0,132,156,91.0,81,89,76.0,51,67
3,Other Walking Issue,64.0,207,323,68.0,107,158,61.0,100,165
4,Cyclist behavior,78.0,87,111,85.0,51,60,71.0,36,51
5,Double parking,80.0,159,199,81.0,21,26,80.0,138,173
6,Accessibility Issue,62.0,90,146,66.0,42,64,59.0,48,82
7,Failure to stop for pedestrians,71.0,432,612,73.0,330,455,65.0,102,157
8,Red light running,82.0,240,294,86.0,186,216,69.0,54,78
9,Blocking the crosswalk,83.0,101,121,89.0,75,84,70.0,26,37
